## Acoustic Integration - Ceiling Sail Orientation

http://localhost:8888/?token=sloth

In [ ]:
if "mi" not in vars():
    from tqdm import trange

    import torch
    import numpy as np
    import matplotlib.pyplot as plt

    import drjit as dr
    import mitsuba as mi

    from libs import utils

    plt.style.use('ggplot')

    mi.set_variant('cuda_ad_acoustic')
    mi.set_log_level(mi.LogLevel.Warn)

    sess_seed   = np.random.randint(0, 2**30)
    sess_seed_g = np.random.randint(0, 2**30)
    print(f"session seeds are: sess_seed={sess_seed}; sess_seed_g={sess_seed_g}")

### Scene Construction

In [ ]:
mic_poses  = np.array([[2., 4., 2.],
                       [5., 3., 2.],
                       [8., 2., 2.],
                       [2., 4., 5.],
                       [5., 3., 5.],
                       [8., 2., 5.],
                       [2., 4., 8.],
                       [5., 3., 8.],
                       [8., 2., 8.]])

sail_poses_dict = {}
sail_size  = np.array([  3.,  2., 1.]) / 2.
# sail_size  = np.array([  3.,  2., 2.]) / 2.
sail_poses = np.array([[ 2.5, 16., 2.],
                       [ 6.5, 16., 2.],
                       [10.5, 16., 2.],
                       [14.5, 16., 2.],
                       [ 2.5, 16., 5.],
                       [ 6.5, 16., 5.],
                       [10.5, 16., 5.],
                       [14.5, 16., 5.],
                       [ 2.5, 16., 8.],
                       [ 6.5, 16., 8.],
                       [10.5, 16., 8.],
                       [14.5, 16., 8.]])

config = {
    "box_dim":        [22., 18., 10.],
    "mic_pos":        mic_poses[0],
    "speaker_pos":    [18.,  2., 5.],
    "speaker_radius": 1.0,

    "absorption": 0.5,
    "scattering": 0.5,

    "wav_bins":  mic_poses.shape[0],
    "time_bins":   10,
    "max_time":  0.20,

    "integrator": "prb_reparam_acoustic",
    "max_depth": 3,
    "spp": 2**22,
}

tf      = mi.ScalarTransform4f
box_dim = np.array(config['box_dim']) / 2.
time    = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)

# config["max_depth"] = utils.estimate_max_depth(config["box_dim"], config["max_time"], 1.5)
print(f"max_depth = {config['max_depth']}")

In [ ]:
scene_dict = utils.shoebox_scene(**config)

scene_dict["integrator"]["skip_direct"] = False
# scene_dict["sensor"]["film"]["rfilter"] = {
#     "type": "gaussian",
#     "stddev": 0.3,
# }

del scene_dict["sensor"]["microphoneA"]
for i, m in enumerate(mic_poses):
    scene_dict["sensor"][f"microphone_{i}"] = {
        "type": "microphone",
        # "to_world": tf.translate(m - box_dim),
        "cos_cutoff": 0.0,
        "to_world": tf.look_at(
            origin=(m - box_dim),
            target=(np.mean(sail_poses, axis=0) - box_dim),
            up=[0, 0, 1]
        ),
    }

sail_vertex_base = mi.Transform4f.scale(sail_size).rotate([1., 0., 0.], angle=90.) @ mi.Point3f(
    [-1., -1.,  1.,  1.],
    [-1.,  1., -1.,  1.],
    [ 0.,  0.,  0.,  0.]
)

for i, s in enumerate(sail_poses):
    sail_poses_dict[f"sail_{i}.vertex_positions"] = mi.Point3f(s - box_dim)
    scene_dict[f"sail_{i}"] = {
        "type": "ply",
        "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/rectangle.ply",
        "bsdf": {
            "type": "acousticbsdf",
            "scattering": { "type": "spectrum", "value": 0.1 },
            "absorption": { "type": "spectrum", "value": 0.2 },
        },
        "to_world": tf.translate(s - box_dim).scale(sail_size).rotate([1., 0., 0.], angle=90.),
    }

### Optimization Setup

In [ ]:
opt = mi.ad.Adam(lr=0.002)
opt["sail_0.vertex_positions"]  = mi.Point2f(-0.4,  0.7)
opt["sail_1.vertex_positions"]  = mi.Point2f(-0.4,  0.4)
opt["sail_2.vertex_positions"]  = mi.Point2f(-0.4, -0.2)
opt["sail_3.vertex_positions"]  = mi.Point2f(-0.4, -0.5)

opt["sail_4.vertex_positions"]  = mi.Point2f( 0.0,  0.7)
opt["sail_5.vertex_positions"]  = mi.Point2f( 0.0,  0.4)
opt["sail_6.vertex_positions"]  = mi.Point2f( 0.0, -0.2)
opt["sail_7.vertex_positions"]  = mi.Point2f( 0.0, -0.5)

opt["sail_8.vertex_positions"]  = mi.Point2f( 0.4,  0.7)
opt["sail_9.vertex_positions"]  = mi.Point2f( 0.4,  0.4)
opt["sail_10.vertex_positions"] = mi.Point2f( 0.4, -0.2)
opt["sail_11.vertex_positions"] = mi.Point2f( 0.4, -0.5)

# opt["sail_0.vertex_positions"]  = mi.Point2f(-0.34,  0.69)
# opt["sail_1.vertex_positions"]  = mi.Point2f(-0.34,  0.51)
# opt["sail_2.vertex_positions"]  = mi.Point2f(-0.34,  0.64)
# opt["sail_3.vertex_positions"]  = mi.Point2f(-0.34, -0.31)

# opt["sail_4.vertex_positions"]  = mi.Point2f( 0.55,  0.68)
# opt["sail_5.vertex_positions"]  = mi.Point2f( 0.01,  0.51)
# opt["sail_6.vertex_positions"]  = mi.Point2f(-0.05,  0.06)
# opt["sail_7.vertex_positions"]  = mi.Point2f( 0.00, -0.32)

# opt["sail_8.vertex_positions"]  = mi.Point2f( 0.34,  0.69)
# opt["sail_9.vertex_positions"]  = mi.Point2f( 0.35,  0.51)
# opt["sail_10.vertex_positions"] = mi.Point2f( 0.38,  0.06)
# opt["sail_11.vertex_positions"] = mi.Point2f( 0.40, -0.31)

# V = np.load("../data/ceiling-sail/ceiling-sail-05.npy")
# V = np.random.rand(1, 1, sail_poses.shape[0], 2) * 0.6 - 0.3
# for i, key in enumerate(sail_poses_dict):
    # opt[key] = mi.Point2f(V[0, -1, i])
    # opt[key] = mi.Point2f(0.0, 0.0)

def apply_transform(params_to_update, optim):
    for key in sail_poses_dict:
        optim[key] = dr.clamp(optim[key], -1., 1.)
        transf = mi.Transform4f.translate(sail_poses_dict[key]) \
                               .rotate(axis=[1., 0., 0.], angle=(optim[key].x * 30.)) \
                               .rotate(axis=[0., 0., 1.], angle=(optim[key].y * 30.))
        params_to_update[key] = dr.ravel(transf @ sail_vertex_base)
    params_to_update.update()

In [ ]:
@dr.wrap_ad(source='drjit', target='torch')
def loss(hist, ref=None):
    assert ref is None
    # f = torch.ones(hist.shape[1], device=hist.device)
    # f[4] = -1
    # return torch.sum(hist[:, :, 0] * f[None, :])
    # return torch.mean(torch.square(hist[:, 0, 0] - hist[:, 1, 0]))
    return torch.var(torch.mean(hist[:, :, 0], dim=0))

### Visualization

In [ ]:
visualize = False
if visualize:
    scene_dict_vis = utils.shoebox_scene_visual(resf=6, **config)

    scene_dict_vis["sensor"]["film"]["width"]  = 200 * 4
    scene_dict_vis["sensor"]["film"]["height"] = 100 * 3
    scene_dict_vis["sensor"]["cameraA"]["to_world"] = tf.look_at(
        origin=np.array([-50., -5., 10.]),
        target=np.zeros(3),
        up=[0, 1, 0]
    )
    scene_dict_vis["sensor"]["cameraB"] = {
        "type": "perspective",
        "to_world": tf.look_at(
            origin=np.array([10., -2., 50.]),
            target=np.zeros(3),
            up=[0, 1, 0]
        ),
    }

    del scene_dict_vis["microphoneA"]
    for i, m in enumerate(mic_poses):
        scene_dict_vis[f"microphone_{i}"] = {
            "type": "sphere",
            "radius": .25,
            "to_world": tf.translate(m - box_dim),
            "emitter": { "type": "area", "radiance": { "type": "rgb", "value": [.4, .0, .1] } },
        }

    sail_colors = [
        { "type": "rgb", "value": [0.8863, 0.2902, 0.2000] },
        { "type": "rgb", "value": [0.4667, 0.4667, 0.4667] },
        { "type": "rgb", "value": [0.9843, 0.7569, 0.3686] },
        { "type": "rgb", "value": [0.5569, 0.7294, 0.2588] },
        { "type": "rgb", "value": [0.8863, 0.2902, 0.2000] },
        { "type": "rgb", "value": [0.4667, 0.4667, 0.4667] },
        { "type": "rgb", "value": [0.9843, 0.7569, 0.3686] },
        { "type": "rgb", "value": [0.5569, 0.7294, 0.2588] },
        { "type": "rgb", "value": [0.8863, 0.2902, 0.2000] },
        { "type": "rgb", "value": [0.4667, 0.4667, 0.4667] },
        { "type": "rgb", "value": [0.9843, 0.7569, 0.3686] },
        { "type": "rgb", "value": [0.5569, 0.7294, 0.2588] },
    ]

    assert len(sail_colors) == sail_poses.shape[0]

    for i, s in enumerate(sail_poses):
        scene_dict_vis[f"sail_{i}"] = {
            "type": "ply",
            "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/rectangle.ply",
            "bsdf": { "type": "diffuse", "reflectance": sail_colors[i] },
            "to_world": tf.translate(s - box_dim).scale(sail_size).rotate([1., 0., 0.], angle=90.)
        }

In [ ]:
if visualize:
    mi.set_variant("cuda_ad_rgb")
    scene  = mi.load_dict(scene_dict_vis)
    params = mi.traverse(scene)
    apply_transform(params, opt)
    img = mi.render(scene, seed=sess_seed)
    utils.plot_img(img)

In [ ]:
generate_process_images = False

if generate_process_images and visualize:
    exi = 5
    V = np.load(f"../data/ceiling-sail/ceiling-sail-{exi:02d}.npy")
    L = np.load(f"../data/ceiling-sail/ceiling-sail-{exi:02d}_losses.npy")
    assert V.shape[1] == L.shape[0]

    L_min, L_max = np.min(L) * 0.99, np.max(L) * 1.01

    for i in trange(V.shape[1]):
        for k in range(V.shape[2]):
            opt[f"sail_{k}.vertex_positions"] = mi.Point2f(V[0, i, k])
        apply_transform(params, opt)

        img = mi.render(scene, seed=sess_seed)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))

        ax1.plot(L[:i])
        ax1.scatter(i, L[i])
        ax1.set_xlim(-5, L.shape[0] + 5)
        ax1.set_ylim(L_min, L_max)
        ax1.set_yscale("log")
        ax1.set_xlabel("iteration")
        ax1.set_ylabel("loss")

        for j in range(V.shape[2]):
            ax2.plot(V[0, :i, j, 0], V[0, :i, j, 1])
        ax2.quiver( V[0, i, :, 0],  V[0, i, :, 1],
                   -V[1, i, :, 0], -V[1, i, :, 1], color='k')

        ticks = (np.linspace(-1., 1., 7), np.linspace(-30., 30., 7))
        ax2.set_xticks(*ticks)
        ax2.set_yticks(*ticks)
        ax2.set_xlim(-0.9, 0.9)
        ax2.set_ylim(-0.9, 0.9)
        ax2.set_xlabel("$R_x$ in degrees")
        ax2.set_ylabel("$R_z$ in degrees")

        fig.tight_layout()
        fig.canvas.draw()
        w, h = fig.canvas.get_width_height()
        plot = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(h, w, 4)[:, :, :3]
        plt.close()
        # plt.show()

        img_conv = mi.TensorXf(utils.bitmap(img, convert=True)).numpy().astype(np.uint8)
        img_merg = np.vstack([img_conv, plot])
        mi.util.write_bitmap(f"../data/ceiling-sail/images/image_{i:03d}.png", img_merg)

In [ ]:
if visualize:
    assert False # stop here if "run all" is hit

### Load and Render Acoustic Scene

In [ ]:
scene  = mi.load_dict(scene_dict)
params = mi.traverse(scene)
apply_transform(params, opt)

img   = mi.render(scene, seed=sess_seed)
utils.plot_hist(mi.TensorXf(img.numpy()[:, :, 0]), **config)

### Compare neutral to optimized position

In [ ]:
compare = False
if compare:
    opt["sail_0.vertex_positions"]  = mi.Point2f(-0.4,  0.7)
    opt["sail_1.vertex_positions"]  = mi.Point2f(-0.4,  0.4)
    opt["sail_2.vertex_positions"]  = mi.Point2f(-0.4, -0.2)
    opt["sail_3.vertex_positions"]  = mi.Point2f(-0.4, -0.5)

    opt["sail_4.vertex_positions"]  = mi.Point2f( 0.0,  0.7)
    opt["sail_5.vertex_positions"]  = mi.Point2f( 0.0,  0.4)
    opt["sail_6.vertex_positions"]  = mi.Point2f( 0.0, -0.2)
    opt["sail_7.vertex_positions"]  = mi.Point2f( 0.0, -0.5)

    opt["sail_8.vertex_positions"]  = mi.Point2f( 0.4,  0.7)
    opt["sail_9.vertex_positions"]  = mi.Point2f( 0.4,  0.4)
    opt["sail_10.vertex_positions"] = mi.Point2f( 0.4, -0.2)
    opt["sail_11.vertex_positions"] = mi.Point2f( 0.4, -0.5)
    # for k in range(sail_poses.shape[0]):
    #     opt[f"sail_{k}.vertex_positions"] = mi.Point2f(0., 0.)

    apply_transform(params, opt)
    img_ref = mi.render(scene, seed=sess_seed)

    V = np.load("../data/ceiling-sail/ceiling-sail-05.npy")
    for k in range(sail_poses.shape[0]):
        opt[f"sail_{k}.vertex_positions"] = mi.Point2f(V[0, -1, k])

    apply_transform(params, opt)
    img = mi.render(scene, seed=sess_seed)

    ############
    # plotting #
    ############

    time  = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), sharey=True, sharex=True)

    ax1.plot(time, img.numpy()[:, :, 0])
    ax2.plot(time, img_ref.numpy()[:, :, 0])

    ax1.set_title(f"Optimized, loss={loss(img).numpy()[0]:.1f}")
    ax2.set_title(f"Reference, loss={loss(img_ref).numpy()[0]:.1f}")

    ax1.set_xlabel('time [s]')
    ax2.set_xlabel('time [s]')

    # fig.tight_layout()
    # fig.savefig('reprays48.png')
    fig.show()

### Visualize Gradient-Spectogram

In [ ]:
apply_transform(params, opt)
img = mi.render(scene, seed=sess_seed, seed_grad=sess_seed_g)

dr.enable_grad(img)
l = loss(img)
dr.backward(l)
g = mi.TensorXf(dr.grad(img))

print(dr.min(g), dr.max(g))
utils.plot_hist(g[:, :, 0], log=False, **config)

### Main Optimization Loop

In [ ]:
%matplotlib ipympl

vals, losses, grads = [], [], []

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.set_title("losses")
ax1.set_xlim(-1, 51)

ax2.set_title("values")
ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1);

In [ ]:
iters = 200
if iters > 1:
    n  = len(vals) + iters

for i in trange(iters):
    apply_transform(params, opt)
    img = mi.render(scene, params, seed=sess_seed+i, seed_grad=sess_seed_g+i)

    l = loss(img)
    dr.backward(l, flags=dr.ADFlag.ClearNone if iters < 2 else dr.ADFlag.Default)

    if iters < 2:
        key = "sail_1.vertex_positions"
        display(dr.unravel(mi.Point3f, dr.grad(params[key])))
        display(dr.grad(opt[key]))
    else:
        angles, sail_grads = [], []
        for key in sail_poses_dict:
            angles.append(opt[key])
            sail_grads.append(dr.grad(opt[key]))

        vals.append(angles)
        grads.append(sail_grads)
        losses.append(l)

        opt.step()

        V = np.array(vals)[:, :, 0]
        G = np.array(grads)[:, :, 0]

        ax1.clear()
        ax1.set_title("losses")
        ax1.set_xlim(-n * 0.02, n * 1.02)

        ax1.plot(np.array(losses)[:, 0])

        ax2.clear()
        ax2.set_title("values")
        ax2.plot(V[:, :, 0], V[:, :, 1])
        ax2.set_xlim(-1.1, 1.1)
        ax2.set_ylim(-1.1, 1.1)

        ax2.quiver( V[-1, :, 0],  V[-1, :, 1],
                   -G[-1, :, 0], -G[-1, :, 1])

        fig.canvas.draw()

In [ ]:
if iters > 1:
    V = np.stack([V, G])
    for key in sail_poses_dict:
        print(opt[key])

    # np.save("../data/ceiling-sail/ceiling-sail-06.npy", V)

### Loss Heatmap per Parameter

+ per parameter = per set of two angles describing the orientation of one ceiling sail

In [ ]:
raise Exception("rework")

if False:
    exi = 19
    V = np.load(f"../data/ceiling-sail/ceiling-sail_{exi:02d}.npy")
    G = np.load(f"../data/ceiling-sail/ceiling-sail_{exi:02d}_grads.npy") * -1.
    vals   = np.linspace(-0.5, 0.5, 21, endpoint=True)
    a, b   = np.meshgrid(vals, vals)
    losses = np.zeros(sail_poses.shape[:1] + a.shape)
    print(vals)

    it = 150
    assert it < V.shape[0]

    fig, axis = plt.subplots(2, 2, figsize=(14, 12), sharex=True, sharey=True)
    axis_f = axis.flatten()

    for k in range(losses.shape[0]):
        opt["sail_0.vertex_positions"] = mi.Point2f(V[it, 0])
        opt["sail_1.vertex_positions"] = mi.Point2f(V[it, 1])
        opt["sail_2.vertex_positions"] = mi.Point2f(V[it, 2])
        opt["sail_3.vertex_positions"] = mi.Point2f(V[it, 3])

        for i in range(a.shape[0]):
            for j in range(a.shape[0]):
                # opt[f"sail_{k}.vertex_positions"] = mi.Point2f(V[-1, k]) + mi.Point2f(a[i, j], b[i, j])
                opt[f"sail_{k}.vertex_positions"] = mi.Point2f(a[i, j], b[i, j])
                apply_transform(params)

                img = mi.render(scene, seed=sess_seed)[:, :, 0].numpy()
                l   = np.mean(np.square(img[:, :2] - img[:, 2:]))
                losses[k, i, j] = l

        img = axis_f[k].imshow(losses[k], norm="log", cmap="inferno", interpolation="none", origin="lower")

        axis_f[k].set_xticks(np.arange(vals.shape[0])[::4], np.round(vals, 2)[::4] * 30.)
        axis_f[k].set_yticks(np.arange(vals.shape[0])[::4], np.round(vals, 2)[::4] * 30.)
        axis_f[k].set_title(f"ceiling sail {k}")

        pos  = (vals.shape[0] - 1) * (V[:, k] + 1.0) / 2.
        grad = (vals.shape[0] - 1) * (G[:, k] + 1.0) / 2.
        axis_f[k].plot(pos[:it+1, 0], pos[:it+1, 1], c="C1", linewidth=3)
        axis_f[k].quiver(pos[it, 0], pos[it, 1], grad[it, 0], grad[it, 1], color="C1", width=0.03)

    axis[1, 0].set_xlabel("$R_x$")
    axis[1, 1].set_xlabel("$R_x$")
    axis[0, 0].set_ylabel("$R_z$")
    axis[1, 0].set_ylabel("$R_z$")
    fig.colorbar(img, ax=axis[:, :], fraction=0.1, shrink=0.5).set_label('loss')
    fig.suptitle(f"Loss Heatmap after {it} iterations")
    fig.savefig(f"../data/ceiling-sail/ceiling-sail_{exi:02d}-heatmap-it{it:03d}.png")
    fig.show()

### Plot Optimization Process

In [ ]:
raise Exception("rework")

V1_exi = 7
V2_exi = 20

V1 = np.load(f"../data/ceiling-sail/ceiling-sail_{V1_exi:02d}.npy")
V2 = np.load(f"../data/ceiling-sail/ceiling-sail_{V2_exi:02d}.npy")

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(12, 12), sharey=True)
xlim  = np.max([V1.shape[0], V2.shape[0]])
ticks = (np.linspace(-1., 1., 7), np.linspace(-30., 30., 7))

ax1.set_title(f"V{V1_exi:02d}")
ax1.set_xlim(-1.1, 1.1)
ax1.set_ylim(-1.1, 1.1)
ax1.set_xticks(*ticks)
ax1.set_yticks(*ticks)

ax3.set_yticks(*ticks)
ax3.set_xlim(-5, xlim + 5)

ax5.set_xlim(-5, xlim + 5)
ax5.set_yticks(*ticks)

ax2.set_title(f"V{V2_exi:02d}")
ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1)
ax2.set_xticks(*ticks)

ax4.set_xlim(-5, xlim + 5)
ax6.set_xlim(-5, xlim + 5)

colors = ['C0', 'C3', 'C4', 'C5']
for j, c in enumerate(colors):
    ax1.plot(V1[:, j, 0], V1[:, j, 1], c=c)
    ax3.plot(V1[:, j, 0], c=c)
    ax5.plot(V1[:, j, 1], c=c)
    ax2.plot(V2[:, j, 0], V2[:, j, 1], c=c)
    ax4.plot(V2[:, j, 0], c=c)
    ax6.plot(V2[:, j, 1], c=c)

ax1.scatter(V1[-1, :, 0], V1[-1, :, 1], color=colors)
ax2.scatter(V2[-1, :, 0], V2[-1, :, 1], color=colors)

plt.show()

### Estimated (Forward) Gradients vs. Finite Differences

In [ ]:
raise Exception("rework")

k = 2
g = mi.Point2f(1.0, 0.0)

V = np.load("../data/ceiling-sail/ceiling-sail_07.npy")
opt["sail_0.vertex_positions"] = mi.Point2f(V[-1, 0])
opt["sail_1.vertex_positions"] = mi.Point2f(V[-1, 1])
opt["sail_2.vertex_positions"] = mi.Point2f(V[-1, 2])
opt["sail_3.vertex_positions"] = mi.Point2f(V[-1, 3])
apply_transform(params)

# forward_pass
assert dr.grad_enabled(opt[key])
dr.set_grad(opt[f"sail_{k}.vertex_positions"], g)

img  = mi.render(scene, params, seed=sess_seed, seed_grad=sess_seed_g)
grad = dr.forward_to(img)

# finite differences
img = mi.render(scene, seed=sess_seed)

opt[f"sail_{k}.vertex_positions"] += g * 1e-2
apply_transform(params)
fdiff = mi.render(scene, seed=sess_seed) - img

In [ ]:
raise Exception("rework")

time  = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)
a1, a2 = config["absorption"][0][1], config["absorption"][1][1]
label = [f"$\\alpha={a1:.1f}, m_1$",
         f"$\\alpha={a2:.1f}, m_1$",
         f"$\\alpha={a1:.1f}, m_2$",
         f"$\\alpha={a2:.1f}, m_2$"]

fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(16, 8), sharex=True)

ax1.plot(time, img[:, :, 0].numpy(), label=label)
ax2.plot(time, grad[:, :, 0].numpy(), label=label)
ax3.plot(time, fdiff[:, :, 0].numpy(), label=label)
ax4.plot(time, (img[:, :2, 0]  - img[:, 2:, 0]).numpy())
ax5.plot(time, (grad[:, :2, 0] - grad[:, 2:, 0]).numpy())
ax6.plot(time, (fdiff[:, :2, 0]  - fdiff[:, 2:, 0]).numpy())

ax1.set_ylabel('Energy [a.U.]')
ax1.set_title('histogram')
ax2.set_title('PRB Reparam Grad')
ax3.set_title('Finite Differences')
ax3.legend()

ax4.set_xlabel('time [s]')
ax4.set_ylabel('Difference [a.U.]')
ax5.set_xlabel('time [s]')
ax6.set_xlabel('time [s]')

fig.tight_layout()
# fig.savefig('reprays48.png')
fig.show()

### Loss per optimization step

In [ ]:
fname = "../data/ceiling-sail/ceiling-sail-06.npy"
# V = np.load(fname)
losses = []

for i in trange(V.shape[1]):
    for k in range(V.shape[2]):
        opt[f"sail_{k}.vertex_positions"] = mi.Point2f(V[0, i, k])
    apply_transform(params, opt)

    img = mi.render(scene, seed=sess_seed)
    losses.append(loss(img))

# np.save(fname.replace('.npy', '_losses.npy'), np.array(losses)[:, 0])

f, ax = plt.subplots(1, 1)
ax.plot(np.array(losses)[:, 0])
plt.show()

### Average distance/time emitter-sail-receiver reflection

In [ ]:
em_pos      = np.array(config["speaker_pos"])
em_dist     = np.linalg.norm(sail_poses - em_pos,       axis=1)
for i, m in enumerate(mic_poses):
    print(i, (np.linalg.norm(sail_poses - m, axis=1) + em_dist) / 343.)
print("d", np.linalg.norm(mic_poses - em_pos,        axis=1) / 343.)